In [1]:
import glob
import xlrd
import imageio.v2 as imageio
import numpy as np
import matplotlib.pyplot as plt
import laserbeamsize as lbs
import cv2
import PIL
import matplotlib.pyplot as plt 
import numpy as np
import glob
from scipy import stats~
from PIL import Image
from scipy import optimize
import pandas as pd
import glob
from PIL import Image
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
import math
from sklearn.metrics import r2_score
import pandas as pd
import csv

def create_dataset(datapath):


    img_data_array=[]
    image_list0 = glob.glob(datapath)
    image_list0=image_list0[15000:]
#there are 20000 images in file, however due to the limited memory, only 5000 are used to create the dataset    
              
    a = np.zeros((len(image_list0),600,600))
    for i in range (len(image_list0)):

        img=cv2.imread(image_list0[i])
        image_array = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        image_array=np.array(image_array)
        a[i,:,:] = image_array
    img_data_array=a
    return img_data_array

def create_scoreset(datapath,scorepath):
    
    def read_score(scorepath):

        file = open(scorepath)
        csvreader = csv.reader(file)
        rows = []
        for row in csvreader:
                rows.append(row)
        return rows

    class_name=[]
    image_list0 = glob.glob(datapath)
    
    def gaussian(x, amplitude, mean, stddev):
        return amplitude * np.exp(-((x - mean) / 4 / stddev)**2)

    def get_number_gaussuam(image_list):

        Score_1=[]

        for i in range (len(image_list)):

            img=cv2.imread(image_list[i])
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            beam=gray
            cx, cy, dx, dy, phi = lbs.beam_size(beam)
            image_array = beam
            x=np.linspace(0,600,600)

            a=int(cx)
            b=int(cy)
            c=5 #box half-thickness
            y_x=0

            for j in range (a-c,a+c):
                y_x=y_x+image_array[j,:]/100 #tricky here since if this number is >255 it becomes 0.
            y_x=(y_x)/c*100

            y_y=0

            for j in range (b-c,b+c):
                y_y=y_y+image_array[:,j]/100
            y_y=((y_y)/c)*100


            popt_x, pcov_x = optimize.curve_fit(gaussian, x, y_x+1,maxfev=500000) # tricky here!!!!! y_x+1
            fitted_y_x=popt_x[0] * np.exp(-((x - popt_x[1]) / 4 / popt_x[2])**2)
            a_x=r2_score(y_x,fitted_y_x)
            popt_y, pcov_y = optimize.curve_fit(gaussian, x, y_y+1,maxfev=500000) # same here!!!! y_y+1
            fitted_y_y=popt_y[0] * np.exp(-((x - popt_y[1]) / 4 / popt_y[2])**2)
            a_y=r2_score(y_y,fitted_y_y)

            s1=abs(a_x*a_y)
            Score_1.append(s1)  

        return Score_1 
    #here use Gaussain fit to give score, other quality analysis method can be replaced here. see quality_number_analysis file.
    s0=get_number_gaussuam(image_list0)  
    outfile = open(scorepath,'w')
    out = csv.writer(outfile)
    out.writerows(map(lambda x: [x], s0))
    outfile.close()
    
    def good_or_bad(scorepath):
        strs = []
        for i in range (len(read_score(scorepath))):
            if read_score(scorepath)[i]>=['0.9']:
                x=1 #good
            else:
                x=0 #bad
                
            strs.append(x)

        return strs

    class_name=good_or_bad(scorepath)
    class_name = np.array(class_name)
    
    return class_name
# extract the image array and class name
X=create_dataset(r'/home/zw3721/Downloads/2022_master_project/M7/*.png')
y=create_scoreset(r'/home/zw3721/Downloads/2022_master_project/M7/*.png','/home/zw3721/Downloads/2022_master_project/s4_gaussian_test.csv')


In [2]:
from sklearn.model_selection import train_test_split
# flatten the images
n_samples = len(X)
data = X.reshape((n_samples, -1))
X_train, X_test, y_train, y_test = train_test_split(data, y , test_size=0.25, random_state=0)
#split to get the train data and test data.

models = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy, precision, recall = {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    
import pandas as pd

df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()

df_model
#get the accuracy, precision and recall of our binary classifiers.

In [4]:
Xtest=create_dataset(r'/home/zw3721/Downloads/2022_master_project/compare images/*.png')

n_samples1 = len(Xtest)
data1 = Xtest.reshape((n_samples1, -1))

print(data1.shape)


predictions = models[key].predict(data1)

print(predictions)

#use our binary classifiers to do the predictions

In [5]:
#CNN trained
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Reshape
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=0)
model = Sequential()
model.add(Reshape((600,600,1), input_shape=(600,600)))
model.add(Conv2D(5, 5, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(10, 5, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(20, 5, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(30, 5, activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
predictions = model.predict(X_test)
#prediction will give the scores and pred give us the good or bad
pred = []
for i in range (len(predictions)):
    if predictions[i]>=np.array(0.9):
        x=1 #good
    else:
        x=0 #bad

    pred.append(x)
    
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred)
# print(cm)
# print(cm[1,0])
# print(cm.ravel())

TN=cm[0,0]
FP=cm[0,1]
FN=cm[1,0]
TP=cm[1,1]
# TN, FP, FN, TP = confusion_matrix(y_test, predictions).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP + TN) / (TP + FP + TN + FN)

print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))

In [6]:
Xtest=create_dataset(r'/home/zw3721/Downloads/2022_master_project/compare images/*.png')

# n_samples1 = len(Xtest)
# data1 = Xtest.reshape((n_samples1, -1))

# print(data1.shape)

predictions = model.predict(Xtest)

print(predictions)

#use CNN to do the predictions